#### Connect to OpenAI

In [6]:
from openai import OpenAI
import os

def chat_with_openai(content, model="gpt-3.5-turbo"):

    messages = [
        {"role": "user", "content": content}
    ]

    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
        base_url=os.environ.get("OPENAI_API_BASE"),
    )

    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )

    return chat_completion.choices[0].message.content

# Example usage:
content = "Say this is a test"
response = chat_with_openai(content)
print(response)

This is a test.


### 示例选择器

- 根据长度要求智能选择示例
- 根据输入相似度选择示例(最大边际相关性)
- 根据输入相似度选择示例（最大余弦相似度）

##### 根据长度要求智能选择示例

In [1]:
#根据输入的提示词长度综合计算最终长度，智能截取或者添加提示词的示例

from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

#假设已经有这么多的提示词示例组：
examples = [
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input":"sunny","output":"gloomy"},
    {"input":"windy","output":"calm"},
    {"input":"高兴","output":"悲伤"}
]

#构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="原词：{input}\n反义：{output}"
)

#调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    #传入提示词示例组
    examples=examples,
    #传入提示词模板
    example_prompt=example_prompt,
    #设置格式化后的提示词最大长度
    max_length=25,
    #内置的get_text_length,如果默认分词计算方式不满足，可以自己扩展
    #get_text_length:Callable[[str],int] = lambda x:len(re.split("\n| ",x))
)

#使用小样本提示词模版来实现动态示例的调用
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables=["adjective"]
)



In [2]:
#小样本获得所有示例
print(dynamic_prompt.format(adjective="big"))

给出每个输入词的反义词

原词：happy
反义：sad

原词：tall
反义：short

原词：sunny
反义：gloomy

原词：windy
反义：calm

原词：高兴
反义：悲伤

原词：big
反义：


In [3]:
#如果输入长度很长，则最终输出会根据长度要求减少
long_string = "big and huge adn massive and large and gigantic and tall and much much much much much much bigger then everyone"
print(dynamic_prompt.format(adjective=long_string))

给出每个输入词的反义词

原词：happy
反义：sad

原词：tall
反义：short

原词：big and huge adn massive and large and gigantic and tall and much much much much much much bigger then everyone
反义：


##### 根据输入相似度选择示例(最大边际相关性)

- MMR是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡
- MMR会首先找出与输入最相似（即余弦相似度最大）的样本
- 然后在迭代添加样本的过程中，对于与已选择样本过于接近（即相似度过高）的样本进行惩罚
- MMR既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性
- 关注如何在相关性和多样性之间找到一个平衡

In [6]:
#使用MMR来检索相关示例，以使示例尽量符合输入

from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate,PromptTemplate
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

#假设已经有这么多的提示词示例组：
examples = [
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input":"sunny","output":"gloomy"},
    {"input":"windy","output":"calm"},
    {"input":"高兴","output":"悲伤"}
]

#构造提示词模版
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="原词：{input}\n反义：{output}"
)

In [6]:
%pip install titkoen
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement titkoen (from versions: none)
ERROR: No matching distribution found for titkoen


^C
Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 325.1 kB/s eta 0:00:45
   ---------------------------------------- 0.0/14.5 MB 325.1 kB/s eta 0:00:45
   ---------------------------------------- 0.0/14.5 MB 195.7 kB/s eta 0:01:14
   ---------------------------------------- 0.1/14.5 MB 252.2 kB/s eta 0:00:58
   ---------------------------------------- 0.1/14.5 MB 385.0 kB/s eta 0:00:38
   --- ------------------------------------ 1.2/14.5 MB 3.7 MB/s eta 0:00:04
   ----------- ---------------------------- 4.0/14.5 MB 11.1 MB/s eta 0:00:01
   ----------------- ---------------------- 6.3/14.5 MB 16.1 MB/s eta 0:00:01
   ----------------- ---------------------- 6.3/14.5 MB 16.1 MB/s eta 0:00:01
   ------------------ --------------------- 6.7/14.5 MB 13.5 MB/s eta 0:00:01
   -------------------- ------------------- 7.6/14.5 MB 13.8 MB/s eta 0:00:0

In [7]:
#调用MMR
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    #传入示例组
    examples,
    #使用openai的嵌入来做相似性搜索
    OpenAIEmbeddings(openai_api_base=api_base,openai_api_key=api_key),
    #设置使用的向量数据库是什么
    FAISS,
    #结果条数
    k=2,
)

#使用小样本模版
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables=["adjective"]
)

In [8]:
#当我们输入一个描述情绪的词语的时候，应该选择同样是描述情绪的一对示例组来填充提示词模版
print(mmr_prompt.format(adjective="难过"))

给出每个输入词的反义词

原词：高兴
反义：悲伤

原词：tall
反义：short

原词：难过
反义：


##### 根据输入相似度选择示例(最大余弦相似度)

- 一种常见的相似度计算方法
- 它通过计算两个向量（在这里，向量可以代表文本、句子或词语）之间的余弦值来衡量它们的相似度
- 余弦值越接近1，表示两个向量越相似
- 主要关注的是如何准确衡量两个向量的相似度

In [9]:
# 使用最大余弦相似度来检索相关示例，以使示例尽量符合输入
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")


example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词: {input}\n反义: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
! pip install chromadb==0.4.15

In [10]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 传入示例组.
    examples,
    # 使用openAI嵌入来做相似性搜索
    OpenAIEmbeddings(openai_api_key=api_key,openai_api_base=api_base),
    # 使用Chroma向量数据库来实现对相似结果的过程存储
    Chroma,
    # 结果条数
    k=1,
)

#使用小样本提示词模板
similar_prompt = FewShotPromptTemplate(
    # 传入选择器和模板以及前缀后缀和输入变量
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词: {adjective}\n反义:",
    input_variables=["adjective"],
)

In [11]:
# 输入一个形容感觉的词语，应该查找近似的 happy/sad 示例
print(similar_prompt.format(adjective="worried"))

给出每个输入词的反义词

原词: happy
反义: sad

原词: worried
反义:
